Handling missing irrigation data

Proxy approach
- Calculate NDMI changes between consecutive readings
- Set thresholds for significant moisture changes
- Identify moisture spikes - check with precipitation levels

In [1]:
import rasterio
import numpy as np
import pandas as pd
from datetime import datetime
import glob
import os

# Read all NDMI files in chronological order
def read_tif_files(directory_path):
    # Get all TIF files
    tif_files = glob.glob(os.path.join(directory_path, '*NDMI*.tif'))
    
    # Extract dates from filenames 
    dates = [os.path.basename(f).split('_to')[0].strip('NDMI_') for f in tif_files] 
    
    # Sort files by date
    sorted_files = sorted(zip(dates, tif_files))
    
    # Store data in a dictionary
    ndmi_data = {}
    for date, file in sorted_files:
        with rasterio.open(file) as src:
            ndmi_data[date] = src.read(1)  # Read the first band
            
    return ndmi_data, src.meta

In [2]:
def calculate_ndmi_changes(ndmi_data):
    dates = sorted(ndmi_data.keys())
    ndmi_changes = {}
    
    for i in range(1, len(dates)):
        current_date = dates[i]
        previous_date = dates[i-1]
        
        change = ndmi_data[current_date] - ndmi_data[previous_date]
        ndmi_changes[current_date] = change
    
    return ndmi_changes

In [3]:
def identify_irrigation_events(ndmi_changes, precipitation_data, threshold=0.1):
    irrigation_events = {}
    
    for date, change in ndmi_changes.items():
        # Get precipitation for this date
        precip = precipitation_data.get(date, 0)
        
        # Create a mask for significant positive changes
        significant_increase = change > threshold
        
        if precip < 5:  # Assuming less than 5mm rain is negligible
            irrigation_mask = significant_increase
            
            irrigation_amount = change[irrigation_mask] * 100  # Convert NDMI change to mm of water
            
            irrigation_events[date] = {
                'area_irrigated': np.sum(irrigation_mask),
                'average_amount': np.mean(irrigation_amount) if len(irrigation_amount) > 0 else 0
            }
    
    return irrigation_events

In [4]:
def analyze_irrigation_patterns(directory_path, weather_csv_path):
    # Read weather data
    weather_data = pd.read_csv(weather_csv_path, parse_dates=['date'])
    weather_dict = weather_data.set_index('date')['precipitation_new'].to_dict()
    
    # Read NDMI data
    ndmi_data, metadata = read_tif_files(directory_path)
    
    # Calculate NDMI changes
    ndmi_changes = calculate_ndmi_changes(ndmi_data)
    
    # Identify irrigation events
    irrigation_events = identify_irrigation_events(ndmi_changes, weather_dict)
    
    # Convert results to DataFrame for analysis
    results_df = pd.DataFrame.from_dict(irrigation_events, orient='index')
    
    return results_df

In [6]:
ndmi_dir = "data/NDMI_Weekly_Exports/NDMI_Weekly_Exports_2019_2024/"
weather_json = "weather_data.json"
import pandas as pd
df = pd.read_json(weather_json)

In [8]:
df['precipitation_new'] = df['precipitation'].apply(lambda x: x['total'])

In [ ]:
def main():
    # Replace with your paths
    ndmi_directory = "path/to/ndmi/files"
    weather_file = "path/to/weather.csv"
    
    results = analyze_irrigation_patterns(ndmi_directory, weather_file)
    
    # Save results
    results.to_csv('irrigation_estimates.csv')
    
    # Basic statistics
    print("Irrigation Summary Statistics:")
    print(results.describe())
    
    # Plot temporal pattern
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(12, 6))
    plt.plot(results.index, results['average_amount'], 'b-', label='Estimated Irrigation')
    plt.title('Estimated Irrigation Events Over Time')
    plt.xlabel('Date')
    plt.ylabel('Estimated Irrigation Amount (mm)')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()